In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict, ClassLabel, Sequence
from transformers import DataCollatorForTokenClassification
import numpy as np
import pandas as pd
import torch
import os


c:\Users\hp\Desktop\telegram_ner_project\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
file_path = "ner_data.conll"  # Make sure this file is in the same folder as your notebook


In [9]:
tokens, ner_tags = read_conll(file_path)


NameError: name 'read_conll' is not defined

In [12]:
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)


In [5]:
model_checkpoint = "xlm-roberta-base"

unique_labels = sorted(set(tag for seq in ner_tags for tag in seq))

label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
)



NameError: name 'ner_tags' is not defined

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [16]:
from datasets import Dataset

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True, truncation=True)
    labels = []

    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token, ignored by loss function
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                # For wordpieces inside the same word, assign the same label
                label_ids.append(label2id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Create dataset and split into train and test
dataset = Dataset.from_dict({"tokens": tokens, "ner_tags": ner_tags})
dataset = dataset.train_test_split(test_size=0.2)

# Apply tokenizer & align labels
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map: 100%|██████████| 634/634 [00:00<00:00, 2000.98 examples/s]


In [10]:
# Assuming ner_tags is already loaded from read_conll()
unique_labels = sorted(set(tag for seq in ner_tags for tag in seq))

label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}


NameError: name 'ner_tags' is not defined

In [18]:
import transformers
print(transformers.__version__)


4.52.4


In [4]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

model_checkpoint = "xlm-roberta-base"

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
)

data_collator = DataCollatorForTokenClassification(tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2
)


NameError: name 'unique_labels' is not defined